In [1]:
import pandas as pd
import numpy as np

In [8]:
combined_df = pd.read_csv('./big_data_ignore/april_may_combined.csv', low_memory=False)

In [3]:
clean_april = pd.read_csv('./big_data_ignore/clean_april.csv', low_memory=False)
clean_may = pd.read_csv('./big_data_ignore/clean_may.csv', low_memory=False)

In [4]:
complaint_ix = pd.read_csv('./big_data_ignore/2016-06-03-complaint-index.csv', encoding='latin-1')
media_ix = pd.read_csv('./big_data_ignore/2016-06-03-all-media-files-index.csv', encoding='latin-1')                

complaint_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)
media_ix.rename(columns={'COMPLAINT_NUMBER': 'complaint_number'}, inplace=True)

In [16]:
april_nums = clean_april.complaint_number.dropna().unique().tolist()
may_nums = clean_may.complaint_number.dropna().unique().tolist()
cix_nums = complaint_ix.complaint_number.dropna().unique().tolist()
mix_nums = media_ix.complaint_number.dropna().unique().tolist()
combined_nums = combined_df.complaint_number.dropna().unique().tolist()

len([num for num in cix_nums if num not in combined_nums])

4

In [4]:
complaint_ix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 14 columns):
complaint_number       102 non-null int64
INCIDENT_TYPE          102 non-null object
NOTIFICATION_DATE      102 non-null object
INCIDENT_TIME_START    102 non-null object
SUBJECT_1              102 non-null object
SUBJECT_2              102 non-null object
SUBJECT_3              102 non-null object
SUBJECT_4              102 non-null object
DOCS_WITHHELD_NOTES    102 non-null object
HIPAA_WITHHELD         102 non-null bool
JUVENILE_WITHHELD      102 non-null bool
VIDEO_COUNT            102 non-null int64
AUDIO_COUNT            102 non-null int64
DOCUMENT_COUNT         102 non-null int64
dtypes: bool(2), int64(4), object(8)
memory usage: 9.8+ KB


In [5]:
media_ix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 7 columns):
complaint_number     1553 non-null int64
MEDIA_FILE_TYPE      1553 non-null object
IPRA_PORTAL_LABEL    1553 non-null object
IPRA_URL             1553 non-null object
FILE_PROBLEMS        1553 non-null object
PUBLIC_FILE_NAME     1553 non-null object
PUBLIC_URL           1553 non-null object
dtypes: int64(1), object(6)
memory usage: 85.0+ KB


In [6]:
len(complaint_ix.complaint_number.unique())

102

In [7]:
len(media_ix.complaint_number.unique())

90

In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161256 entries, 0 to 161255
Data columns (total 51 columns):
accused_appointment_date        95782 non-null object
accused_detail                  12364 non-null float64
accused_officer_fname           95782 non-null object
accused_officer_lname           95782 non-null object
accused_position                95770 non-null object
accused_star                    94122 non-null float64
accused_unit                    95770 non-null float64
address                         160851 non-null float64
beat                            161224 non-null float64
complaint_date                  24501 non-null object
complaint_number                161256 non-null int64
converted_rec_i                 24501 non-null object
cr_required                     24501 non-null object
current_category                161215 non-null object
current_category_code           161215 non-null object
current_status                  161256 non-null object
discipline_code

In [17]:
full = pd.merge(combined_df, complaint_ix, on='complaint_number', how='right')
full = pd.merge(full, media_ix, on='complaint_number', how='left')

In [18]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592968 entries, 0 to 592967
Data columns (total 70 columns):
accused_appointment_date        523158 non-null object
accused_detail                  1810 non-null float64
accused_officer_fname           523158 non-null object
accused_officer_lname           523158 non-null object
accused_position                523158 non-null object
accused_star                    518786 non-null float64
accused_unit                    523158 non-null float64
address                         592943 non-null float64
beat                            592943 non-null float64
complaint_date                  409365 non-null object
complaint_number                592968 non-null float64
converted_rec_i                 409365 non-null object
cr_required                     409365 non-null object
current_category                592943 non-null object
current_category_code           592943 non-null object
current_status                  592943 non-null object
disci

In [19]:
cols = [col for col in full.columns if col != 'complaint_number']
group = full.groupby('complaint_number')
df_list = [group[col].unique().reset_index() for col in cols]

In [20]:
summary_df = df_list[0]
for col_df in df_list[1:]:
    summary_df = pd.merge(summary_df, col_df, on='complaint_number')

In [21]:
def clear_nans(x):
    if len(x) == 1:
        if ~(x[0]==x[0]):
            return []
        else:
            return x
    else:
        return x

In [41]:
pd.options.display.max_rows = 100
cols = summary_df.columns[1:]
counts = summary_df[cols].applymap(lambda x: True if len(x)==1 & ~(x[0]==x[0]) else False)\
                .sum(axis=0)/len(summary_df)*100

In [47]:
counts.index[(counts<20).values]

Index(['address', 'beat', 'current_category', 'current_category_code',
       'current_status', 'incident_time_end', 'incident_time_start',
       'initial_category', 'initial_category_code', 'involved_officer_age',
       'involved_officer_fname', 'involved_officer_lname',
       'involved_officer_position', 'involved_officer_race',
       'involved_officer_sex', 'involved_officer_unit',
       'ipra_investigator_firstname', 'ipra_investigator_lastname',
       'location_code', 'notification_date', 'street', 'INCIDENT_TYPE',
       'NOTIFICATION_DATE', 'INCIDENT_TIME_START', 'SUBJECT_1', 'SUBJECT_2',
       'SUBJECT_3', 'SUBJECT_4', 'DOCS_WITHHELD_NOTES', 'HIPAA_WITHHELD',
       'JUVENILE_WITHHELD', 'VIDEO_COUNT', 'AUDIO_COUNT', 'DOCUMENT_COUNT',
       'MEDIA_FILE_TYPE', 'IPRA_PORTAL_LABEL', 'IPRA_URL', 'FILE_PROBLEMS',
       'PUBLIC_FILE_NAME', 'PUBLIC_URL'],
      dtype='object')

In [24]:
combined_df[['police_shooting', 'police_shooting_no']].apply(pd.Series.unique, axis=1)

,police_shooting,police_shooting_no
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [27]:
combined_df.police_shooting.unique()

array([nan, 'Yes', 'No'], dtype=object)

In [41]:
#combined_df.police_shooting_no.unique()

In [38]:
mask = combined_df.initial_category_code.astype(str).apply(lambda x: True if '18' in x[:2] else False).values

In [40]:
#combined_df.initial_category_code[mask]

In [42]:
full.initial_category.unique()

array([nan, 'FIREARM DISCHARGE WITH HITS - HANDGUN',
       'FIREARM DISCHARGE WITH HITS / ON DUTY',
       'EXCESSIVE FORCE / ON DUTY - INJURY', 'MISCELLANEOUS',
       'EXCESSIVE FORCE / ON DUTY - NO INJURY',
       'UNNECESSARY PHYSICAL CONTACT / ON DUTY - NO INJURY',
       'PROPER CARE - INJURY / DEATH',
       'FIREARM DISCHARGE WITH HITS / OFF DUTY',
       'EXCESSIVE FORCE / OFF DUTY - INJURY', 'SHOTS FIRED - NO HITS',
       'CIVIL SUITS - THIRD PARTY ',
       'EXCESSIVE FORCE - USE OF FIREARM / ON DUTY - NO INJURY'], dtype=object)

In [47]:
full.current_category.value_counts()

FIREARM DISCHARGE WITH HITS / ON DUTY                     564819
EXCESSIVE FORCE / ON DUTY - INJURY                         16952
FIREARM DISCHARGE WITH HITS / OFF DUTY                      2530
FIREARM DISCHARGE WITH HITS - HANDGUN                       2321
UNNECESSARY PHYSICAL CONTACT / ON DUTY - NO INJURY          2156
EXCESSIVE FORCE / OFF DUTY - INJURY                         1590
PROPER CARE - INJURY / DEATH                                1460
SHOTS FIRED - NO HITS                                        449
TAKE DOWN (THROWN TO GROUND)                                 216
MISCELLANEOUS                                                156
EXCESSIVE FORCE / ON DUTY - NO INJURY                        148
EXCESSIVE FORCE - USE OF FIREARM / ON DUTY - NO INJURY       126
CIVIL SUITS - THIRD PARTY                                     20
Name: current_category, dtype: int64

In [48]:
sorted(combined_df.columns.tolist())

['accused_appointment_date',
 'accused_detail',
 'accused_officer_fname',
 'accused_officer_lname',
 'accused_position',
 'accused_star',
 'accused_unit',
 'address',
 'beat',
 'complaint_date',
 'complaint_number',
 'converted_rec_i',
 'cr_required',
 'current_category',
 'current_category_code',
 'current_status',
 'discipline_code',
 'fileyr',
 'finding_code',
 'finding_id',
 'iad_ops',
 'incident_time_end',
 'incident_time_start',
 'initial_category',
 'initial_category_code',
 'involved_officer_age',
 'involved_officer_description',
 'involved_officer_detail',
 'involved_officer_fname',
 'involved_officer_lname',
 'involved_officer_position',
 'involved_officer_race',
 'involved_officer_sex',
 'involved_officer_type',
 'involved_officer_unit',
 'ipra_assign_date_time',
 'ipra_closed_date',
 'ipra_investigate_begin_date',
 'ipra_investigate_end_date',
 'ipra_investigator_firstname',
 'ipra_investigator_lastname',
 'ipra_investigator_type',
 'location_code',
 'notification_date',
 '

array([  432.,   433.,  1331.,  1822.,   522.,   312.,   624.,  1122.,
         734.,   821.,   512.,  1132.,   613.,   423.,  1611.,  1022.,
        1834.,   911.,   621.,   123.,   811.,   511.,  1634.,  1113.,
        1933.,   331.,   124.,  2234.,   225.,  1523.,     0.,  1333.,
         113.,   733.,  2212.,   724.,   612.,   334.,  1821.,   934.,
         313.,  1014.,   634.,  1133.,  1213.,   333.,   825.,  1633.,
        2512.,  1013.,  1012.,  2531.,  1024.,  1134.,  1824.,   523.,
         631.,   913.,  1512.,  1524.,   322.,   831.,  1111.,  3100.,
        1522.,   622.,  1711.,   912.,   834.,  1031.,  2515.,   713.,
         623.,   924.,   726.,   921.,  1731.,   531.,  2011.,   321.,
        1115.,   732.,  2533.,   414.,  1733.,   422.,  1323.,  2511.,
        1532.,  2432.,   735.,   632.,   131.,  2012.,   122.,  1712.,
        1121.,   931.,   524.,   614.,   121.,   633.,   224.,   533.,
         725.,  2413.,  2422.,  1114.,  1513.,  1034.,  2424.,  2535.,
      

In [55]:
len(full.accused_officer_lname.unique())

69

In [56]:
len(combined_df.accused_officer_lname.unique())

2268

In [57]:
combined_df['full_name'] = combined_df.accused_officer_lname.astype('str') + '_' + combined_df.accused_officer_fname.astype('str')

In [59]:
len(combined_df.full_name.unique())

3047